In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate

# Token classification

The token classification encompasses any problem that can be formulated as "attributing a label to each token in a sentence".
* **Named entity recognition (NER)**: Find the entities in a sentence. This can be formulated as attributing a label to each token by having one class per entity and one class for "no entity".
* **Part-of-speech tagging (POS)**: Mark each word in a sentence as corresponding to a particular part of speech (such as noun, verb, adjective, etc.).
* **Chunking**: Find the tokens that belong to the same entity. This task (which can be combined with POS or NER) can be formulated as attributing one label (usually `B-`) to any tokens that are at the beginning of a chunk, another label(usually `I-`) to tokens that are inside a chunk, and a third label (usually `O`) to tokens that do not belong to any chunk.

## Preparing the data

### The CoNLL-2003 dataset

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset('conll2003')

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 14041
    })
    validation: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3250
    })
    test: Dataset({
        features: ['id', 'tokens', 'pos_tags', 'chunk_tags', 'ner_tags'],
        num_rows: 3453
    })
})

This dataset contains labels for three tasks: NER, POS, and chunking. In this dataset, the input texts are not presented as sentences or documents, but lists of words (`tokens`).

In [4]:
raw_datasets['train'][0]['tokens']

['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.']

In [5]:
raw_datasets['train'][0]['ner_tags']

[3, 0, 7, 0, 0, 0, 7, 0, 0]

The mapping between those integers and the label names is at the `features` atttribute:

In [6]:
ner_feature = raw_datasets['train'].features['ner_tags']
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC'], id=None), length=-1, id=None)

This column contains elements that are sequences of `ClassLabel`s. The type of the elements of the sequence is in the `feature` attribute of this `ner_feature`,

In [7]:
label_names = ner_feature.feature.names
label_names

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-MISC', 'I-MISC']

We can use this to decode the labels:

In [8]:
words = raw_datasets['train'][0]['tokens']
labels = raw_datasets['train'][0]['ner_tags']

line1 = ''
line2 = ''

for word, label in zip(words, labels):
    full_label = label_names[label]
    max_length = max(len(word), len(full_label))
    line1 += word + ' ' * (max_length - len(word) + 1)
    line2 += full_label + ' ' * (max_length - len(full_label) + 1)

print(line1)
print(line2)

EU    rejects German call to boycott British lamb . 
B-ORG O       B-MISC O    O  O       B-MISC  O    O 


### Processing the data

The texts need to be converted to token IDs before the model can make sense of them.

We will use a BERT pretrained model. Start with the `tokenizer`

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = 'bert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [10]:
tokenizer.is_fast

True

To tokenize a pre-tokenized input, we can use our `tokenizer` with `is_split_into_word=True`:

In [13]:
inputs = tokenizer(raw_datasets['train'][0]['tokens'],
                   is_split_into_words=True)
inputs.tokens()

['[CLS]',
 'EU',
 'rejects',
 'German',
 'call',
 'to',
 'boycott',
 'British',
 'la',
 '##mb',
 '.',
 '[SEP]']

The tokenizer adds the special tokens used by the model.

Note the word `lamb` was tokenized into two subwords, `la` and `##mb`. This introduces a mismatch between our inputs and the labels: the list of labels has only 9 elements, whereas our input now has 12 tokens after tokenization.

Fortunately, we can easily map each token to its corresponding word with the Tokenizers library:

In [14]:
inputs.word_ids()

[None, 0, 1, 2, 3, 4, 5, 6, 7, 7, 8, None]

Now we need to write a function to expand our label list to match the tokens:
* the special tokens get a label of `-100` because by default `-100` is an index that is ignored in the loss function we use (cross entropy).
* each token gets the same label as the token that started the word it's inside, since they are part of the same entity. For tokens inside a word but not at the beginning, we replace the `B-` with `I-` (since the token does not begin with entity):

In [16]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None

    for word_id in word_ids:
        if word_id != current_word:
            # start of a new word
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # special token
            new_labels.append(-100)
        else:
            # same word as previous token
            label = labels[word_id]
            # if the label is B-XXX, we change it to I-XXX
            if label % 2 == 1:
                label += 1

            new_labels.append(label)

    return new_labels

In [17]:
labels = raw_datasets['train'][0]['ner_tags']
word_ids = inputs.word_ids()

print(labels)
print(align_labels_with_tokens(labels, word_ids))

[3, 0, 7, 0, 0, 0, 7, 0, 0]
[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]


To preprocess the whole dataset, we need to write a mapping function:

In [18]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples['tokens'],
                                 truncation=True,
                                 is_split_into_words=True)
    all_labels = examples['ner_tags']
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))

    tokenized_inputs['labels'] = new_labels
    return tokenized_inputs

In [19]:
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets['train'].column_names,
)

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

## Fine-tuning the model with the Trainer API

### Data collation

Since the `DataCollatorWithPadding` only pads the inputs (input IDs, attention mask, and token type IDs), our labels should be padded the exact same way as the inputs so that they stay the same size for the token classification, so we use `DataCollatorForTokenClassification`:

In [20]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

After padding:

In [21]:
batch = data_collator([tokenized_datasets['train'][i] for i in range(2)])
batch['labels']

tensor([[-100,    3,    0,    7,    0,    0,    0,    7,    0,    0,    0, -100],
        [-100,    1,    2, -100, -100, -100, -100, -100, -100, -100, -100, -100]])

Before padding:

In [24]:
for i in range(2):
    print(tokenized_datasets["train"][i]["labels"])

[-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, 0, -100]
[-100, 1, 2, -100]


The second set of labels has been padded to the length of the first one using `-100`s.

### Metrics

The traditional framework used to evaluate token classification prediction is *seqeval*:

In [ ]:
!pip install seqeval

In [26]:
import evaluate

metric = evaluate.load('seqeval')

This metric takes the lists of labels as strings, not integers, so we will need to fully decode the predictions and labels before passing them to the metric.

First, we will get the labels for our first training example:

In [28]:
labels = raw_datasets['train'][0]['ner_tags']
labels = [label_names[i] for i in labels]
labels

['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']

In [29]:
# create fake predictions
predictions = labels.copy()
predictions[2] = 'O'

Then, we can compute the metric

In [30]:
metric.compute(predictions=[predictions], references=[labels])

{'MISC': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'ORG': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 0.6666666666666666,
 'overall_f1': 0.8,
 'overall_accuracy': 0.8888888888888888}

We get the precision, recall, and F1 score for each separate entity, as well as overall.

Now we need to write the `compute_metrics()` function

In [32]:
import numpy as np

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != 100] for label in labels]

    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != 100]
        for prediction, label in zip(predictions, labels)
    ]

    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

    return {
        'precision': all_metrics['overall_precision'],
        'recall': all_metrics['overall_recall'],
        'f1': all_metrics['overall_f1'],
        'accuracy': all_metrics['overall_accuracy'],
    }

### Defining the model

In [33]:
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

In [35]:
from transformers import AutoModelForTokenClassification

model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
# check our model has the right number of labels
model.config.num_labels

9

### Fine-tuning the model

In [39]:
from transformers import TrainingArguments

args = TrainingArguments(
    'bert-finetuned-ner',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [40]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

In [ ]:
trainer.train()

## A custom training loop

### Preparing everything for training

Build the `DataLoader`s from our datasets.

In [41]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

eval_dataloader = DataLoader(
    tokenized_datasets['validation'],
    collate_fn=data_collator,
    batch_size=8,
)

Create a model instance

In [42]:
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Set up an optimizer

In [43]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

Set up the Accelerator

In [44]:
from accelerate import Accelerator

accelerator = Accelerator()

model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Set up a learning scheduler

In [45]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

### Training loop

To simplify its evaluation part, we define a `postprocess()` function that takes predictions and labels and converts them to lists of strings, like our `metric` object:

In [46]:
def postprocess(predictions, labels):
    predictions = predictions.detach().cpu().clone().numpy()
    labels = labels.detach().cpu().clone().numpy()

    # remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p,l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    return true_predictions, true_labels

In [ ]:
from tqdm.auto import tqdm
import torch

output_dir = "bert-finetuned-ner-accelerate"

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in eval_dataloader:
        with torch.no_grad():
            outputs = model(**batch)

        predictions = outputs.logits.argmax(dim=-1)
        labels = batch['labels']

        # necessary to pad predictions and labels for being gathered
        predictions = accelerator.pad_across_processes(predictions, dim=1, pad_index=-100)
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(predictions)
        labels_gathered = accelerator.gather(labels)

        true_predictions, true_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=true_predictions, references=true_labels)

    results = metric.compute()
    print(
        f"epoch {epoch}:",
        {
            key: results[f'overall_{key}']
            for key in ['precision', 'recall', 'f1', 'accuracy']
        }
    )

    # Save
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

```python
accelerator.wait_for_everyone()
unwrapped_model = accelerator.unwrap_model(model)
unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
```
The first line tells all the processes to wait until everyone is at that stage before continuing. Then we grab the `unwrapped_model`, which is the base model we defined. The `accelerator.prepare()` method changes the model to work in distributed training, so it will not have the `save_pretrained()` method; the `accelerator.unwrap_model()` undoes that step. Lastly, we call `save_pretrained()` but tell that method to use `accelerator.save()` instead of `torch.save()`.

## Using the fine-tuned model

In [ ]:
from transformers import pipeline

model_checkpoint = 'bert-finetuned-ner-accelerate'

token_classifier = pipeline(
    'token-classification',
    model=model_checkpoint,
    aggregation_strategy='simple'
)

token_classifier('My name is Bin and I work in Houston, Texas.')